# Import

In [1]:

import os
from pathlib import Path
import h5py
import pandas as pd
import numpy as np
import arpespythontools as arp
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout, interactive_output
import astropy 
from mpl_toolkits.mplot3d import Axes3D
import pickle
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from datetime import datetime
import csv

In [2]:
%run data_loader.ipynb

map_data2 = map_data2
phi = phi
energy = energy
theta = theta
theta_v = theta_v


Loaded data from cache: 20230317_00029_cache.pkl


In [3]:
def angle_to_wavevector_binding(E_pho, work_f, theta, phi):
    #CONVERTS PIXELS TO THETA
    global theta_v
    
    #print(theta_v)
    E_k = E_pho - work_f
    k = 0.512*np.sqrt(E_k)
    theta_rad = np.radians(theta_v)
    phi_rad = np.radians(phi)
    
    #K_x and K_Y 
    k_x = k*np.sin(theta_rad)
    k_y = k*np.sin(phi_rad)
    
    #k_x = np.degrees(k_x)
    #k_y = np.degrees(k_y)
    return k_x, k_y
        
    
global k_x
global k_y

k_x, k_y = angle_to_wavevector_binding(85, 4, theta_v, phi)

# Variables

In [4]:
global emin1, emax2, vmax1, k_x1, k_x2, vmax2, pmin, pmax, vmax, k_y1, k_y2
emin1, emax2, vmax1 = -0.44, -0.41, 16226
k_x1, k_x2, vmax2 = -1, 360, 50000
k_y1, k_y2 = 0, 121
pmin, pmax, vmax = -1, 2, 35480
current_date = datetime.now().strftime("%Y-%m-%d")

In [5]:
file_path = data_file_path

def get_dataset_name(file_path):
    with h5py.File(file_path, "r") as h5_file:
        if "2D_Data" in h5_file:
            dataset_names = list(h5_file["2D_Data"].keys())  # Extract dataset names
            return dataset_names[0]  # Assuming you want the first dataset name
        else:
            return "Group '2D_Data' not found in the file."

dataset_name = str(get_dataset_name(file_path))

print(f"The dataset name is: {dataset_name}")

The dataset name is: Swept_Spectra46


# MDC in the k_x

In [8]:


def add_bounds_to_csv_mdc(k_y1, k_y2, Ev_Lowerbound, Ev_Upperbound, current_date):
    global dataset_name
    main_folder = "ARPES Waterfall Plots"
    os.makedirs(main_folder, exist_ok=True)
    
    subfolder = os.path.join(main_folder, dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    file_name = os.path.join(subfolder, f"{dataset_name}_MDC_k_y_bounds.csv")
    bounds_df = pd.DataFrame({
        "Energy_Lowerbound": [Ev_Lowerbound],
        "Energy_Upperbound": [Ev_Upperbound],
        "k_y_Lowerbound": [k_y1],
        "k_y_Upperbound": [k_y2],
        "Date": [current_date]
    })
    try:
        bounds_df.to_csv(file_name, mode='a', header=not os.path.exists(file_name), index=False)
        print(f"Bounds added to {file_name}")
    except PermissionError:
        print(f"Permission denied: Unable to write to {file_name}. Please check file permissions.")
    except Exception as e:
        print(f"An error occurred: {e}")

def waterfall_plot(k_y1, k_y2, Ev_Lowerbound, Ev_Upperbound, offset, interval, k_x_min, k_x_max):
    energies = np.arange(Ev_Lowerbound, Ev_Upperbound, interval)
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111)
    
    for i, e in enumerate(energies):
        surface = arp.plane_slice(map_data2, energy, e, e) 
        kx_line = arp.line_profile(surface, k_y, k_y1, k_y2)
        if np.max(kx_line) > 0:
            kx_line /= np.max(kx_line)
        else:
            kx_line = np.zeros_like(kx_line)
        
        mask = (k_x >= k_x_min) & (k_x <= k_x_max)
        ax.plot(k_x[mask], kx_line[mask] + i * offset, color='green', label=f"E = {e:.2f}")
    
    ax.set_ylabel("Intensity (Offset)")
    ax.set_xlabel("Wavevector (k_x)")
    ax.set_title("Waterfall Plot of Constant Energy Slices")
    plt.close(fig)
    return fig

def save_waterfall_plot(figure):
    global dataset_name
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)
    
    unique_id = f"k_y1_{k_y1_slider.value:.2f}_k_y2_{k_y2_slider.value:.2f}_E_{Ev_Lowerbound_slider.value:.2f}_{Ev_Upperbound_slider.value:.2f}"
    file_name = os.path.join(subfolder, f"{dataset_name}_MDC_{unique_id}.png")
    
    figure.savefig(file_name, dpi=1200)
    print(f"Plot saved as {file_name}")

def waterfall_plot_save_data(k_y1, k_y2, Ev_Lowerbound, Ev_Upperbound, offset, interval, k_x_min, k_x_max):
    global dataset_name
    energies = np.arange(Ev_Lowerbound, Ev_Upperbound, interval)
    data_dict = {}
    
    for i, e in enumerate(energies):
        surface = arp.plane_slice(map_data2, energy, e, e)
        kx_line = arp.line_profile(surface, k_y, k_y1, k_y2)
        if np.max(kx_line) > 0:
            kx_line /= np.max(kx_line)
        else:
            kx_line = np.zeros_like(kx_line)
        
        mask = (k_x >= k_x_min) & (k_x <= k_x_max)
        data_dict[e] = list(zip(k_x[mask], kx_line[mask] + i * offset))
    
    max_len = max(len(v) for v in data_dict.values())
    header = []
    for e in sorted(data_dict.keys()):
        header.append(f"k_x (E={e:.2f})")
        header.append(f"Intensity (E={e:.2f})")
    
    formatted_data = []
    for row_idx in range(max_len):
        row = []
        for e in sorted(data_dict.keys()):
            pairs = data_dict[e]
            if row_idx < len(pairs):
                row.extend(pairs[row_idx])
            else:
                row.extend(["", ""])
        formatted_data.append(row)
    
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)
    unique_id = f"k_y1_{k_y1:.2f}_k_y2_{k_y2:.2f}_E_{Ev_Lowerbound:.2f}_{Ev_Upperbound:.2f}"
    csv_filename = os.path.join(subfolder, f"{dataset_name}_MDC_{unique_id}.csv")
    
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(formatted_data)
    
    print(f"CSV data saved successfully to {csv_filename}")


# Sliders for the k_x range selection (specific to MDC waterfall plot)
k_x_min_slider = widgets.FloatSlider(value=np.min(k_x), min=np.min(k_x), max=np.max(k_x), step=0.01, description='k_x min')
k_x_max_slider = widgets.FloatSlider(value=np.max(k_x), min=np.min(k_x), max=np.max(k_x), step=0.01, description='k_x max')

k_y1_slider = widgets.FloatSlider(value=-1, min=k_y[0], max=k_y[-1], step=0.01, description='k_y1')
k_y2_slider = widgets.FloatSlider(value=0.5, min=k_y[0], max=k_y[-1], step=0.01, description='k_y2')
Ev_Lowerbound_slider = widgets.FloatSlider(value=-1.22, min=energy[0], max=energy[-1], step=0.01, description='Lower Energy')
Ev_Upperbound_slider = widgets.FloatSlider(value=-0.38, min=energy[0], max=energy[-1], step=0.01, description='Upper Energy')
offset_slider = widgets.FloatSlider(value=0.2, min=0.01, max=2, step=0.01, description='Offset')
interval_slider = widgets.FloatSlider(value=0.02, min=0.001, max=0.5, step=0.01, description='Interval')

save_mdc_plot_button = widgets.Button(description="Save Plot as PNG")
save_mdc_plot_button.on_click(lambda _: save_waterfall_plot(waterfall_plot(
    k_y1_slider.value, k_y2_slider.value,
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value,
    offset_slider.value, interval_slider.value,
    k_x_min_slider.value, k_x_max_slider.value
)))


save_mdc_csv_button = widgets.Button(description="Save CSV of Plot")
save_mdc_csv_button.on_click(lambda _: waterfall_plot_save_data(
    k_y1_slider.value, k_y2_slider.value,
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value,
    offset_slider.value, interval_slider.value,
    k_x_min_slider.value, k_x_max_slider.value
))

add_mdc_bounds_button = widgets.Button(description="Add Bounds to CSV")
add_mdc_bounds_button.on_click(lambda _: add_bounds_to_csv_mdc(
    k_y1_slider.value, k_y2_slider.value,
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value, current_date
))

display(widgets.HBox([add_mdc_bounds_button, save_mdc_plot_button, save_mdc_csv_button]))

display(widgets.interact(
    waterfall_plot,
    k_y1=k_y1_slider,
    k_y2=k_y2_slider,
    Ev_Lowerbound=Ev_Lowerbound_slider,
    Ev_Upperbound=Ev_Upperbound_slider,
    offset=offset_slider,
    interval=interval_slider,
    k_x_min=k_x_min_slider,
    k_x_max=k_x_max_slider
))


interactive(children=(FloatSlider(value=-1.0, description='k_y1', max=1.1926381598324156, min=-1.1926381598324…

<function __main__.waterfall_plot(k_y1, k_y2, Ev_Lowerbound, Ev_Upperbound, offset, interval, k_x_min, k_x_max)>

# MDC in k_y

In [18]:


def add_bounds_to_csv_mdc(k_x1, k_x2, Ev_Lowerbound, Ev_Upperbound, current_date):
    global dataset_name
    main_folder = "ARPES Waterfall Plots"
    os.makedirs(main_folder, exist_ok=True)
    
    subfolder = os.path.join(main_folder, dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    file_name = os.path.join(subfolder, f"{dataset_name}_MDC_k_x_bounds.csv")
    bounds_df = pd.DataFrame({
        "Energy_Lowerbound": [Ev_Lowerbound],
        "Energy_Upperbound": [Ev_Upperbound],
        "k_x_Lowerbound": [k_x1],
        "k_x_Upperbound": [k_x2],
        "Date": [current_date]
    })
    try:
        bounds_df.to_csv(file_name, mode='a', header=not os.path.exists(file_name), index=False)
        print(f"Bounds added to {file_name}")
    except PermissionError:
        print(f"Permission denied: Unable to write to {file_name}. Please check file permissions.")
    except Exception as e:
        print(f"An error occurred: {e}")

def waterfall_plot(k_x1, k_x2, Ev_Lowerbound, Ev_Upperbound, offset, interval, k_y_min, k_y_max):
    energies = np.arange(Ev_Lowerbound, Ev_Upperbound, interval)
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111)

    for i, e in enumerate(energies):
        surface = arp.plane_slice(map_data2.transpose([0, 2, 1]), energy, e, e)  # Slice at constant energy
        ky_line = arp.line_profile(surface, k_x, k_x1, k_x2)
        if np.max(ky_line) > 0:
            ky_line /= np.max(ky_line)
        else:
            ky_line = np.zeros_like(ky_line)
        
        mask = (k_y >= k_y_min) & (k_y <= k_y_max)
        ax.plot(k_y[mask], ky_line[mask] + i * offset, color='green', label=f"E = {e:.2f}")
    
    ax.set_ylabel("Intensity (Offset)")
    ax.set_xlabel("Wavevector (k_y)")
    ax.set_title("Waterfall Plot of Constant Energy Slices")
    plt.close(fig)
    return fig

def save_waterfall_plot(figure):
    global dataset_name
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)
    
    unique_id = f"k_x1_{k_x1_slider.value:.2f}_k_x2_{k_x2_slider.value:.2f}_E_{Ev_Lowerbound_slider.value:.2f}_{Ev_Upperbound_slider.value:.2f}"
    file_name = os.path.join(subfolder, f"{dataset_name}_MDC_{unique_id}.png")
    
    figure.savefig(file_name, dpi=1200)
    print(f"Plot saved as {file_name}")

def waterfall_plot_save_data(k_x1, k_x2, Ev_Lowerbound, Ev_Upperbound, offset, interval, k_y_min, k_y_max):
    global dataset_name
    energies = np.arange(Ev_Lowerbound, Ev_Upperbound, interval)
    data_dict = {}
    
    for i, e in enumerate(energies):
        surface = arp.plane_slice(map_data2.transpose([0, 2, 1]), energy, e, e)
        ky_line = arp.line_profile(surface, k_x, k_x1, k_x2)
        if np.max(ky_line) > 0:
            ky_line /= np.max(ky_line)
        else:
            ky_line = np.zeros_like(ky_line)
        
        mask = (k_y >= k_y_min) & (k_y <= k_y_max)
        data_dict[e] = list(zip(k_y[mask], ky_line[mask] + i * offset))
    
    max_len = max(len(v) for v in data_dict.values())
    header = []
    for e in sorted(data_dict.keys()):
        header.append(f"k_y (E={e:.2f})")
        header.append(f"Intensity (E={e:.2f})")
    
    formatted_data = []
    for row_idx in range(max_len):
        row = []
        for e in sorted(data_dict.keys()):
            pairs = data_dict[e]
            if row_idx < len(pairs):
                row.extend(pairs[row_idx])
            else:
                row.extend(["", ""])
        formatted_data.append(row)
    
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)
    unique_id = f"k_x1_{k_x1:.2f}_k_x2_{k_x2:.2f}_E_{Ev_Lowerbound:.2f}_{Ev_Upperbound:.2f}"
    csv_filename = os.path.join(subfolder, f"{dataset_name}_MDC_{unique_id}.csv")
    
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(formatted_data)
    
    print(f"CSV data saved successfully to {csv_filename}")


k_y_min_slider = widgets.FloatSlider(value=np.min(k_y), min=np.min(k_y), max=np.max(k_y), step=0.01, description='k_y min')
k_y_max_slider = widgets.FloatSlider(value=np.max(k_y), min=np.min(k_y), max=np.max(k_y), step=0.01, description='k_y max')

k_x1_slider = widgets.FloatSlider(value=-1, min=k_x[0], max=k_x[-1], step=0.01, description='k_x1')
k_x2_slider = widgets.FloatSlider(value=0.5, min=k_x[0], max=k_x[-1], step=0.01, description='k_x2')
Ev_Lowerbound_slider = widgets.FloatSlider(value=-1.22, min=energy[0], max=energy[-1], step=0.01, description='Lower Energy')
Ev_Upperbound_slider = widgets.FloatSlider(value=-0.38, min=energy[0], max=energy[-1], step=0.01, description='Upper Energy')
offset_slider = widgets.FloatSlider(value=0.2, min=0.01, max=2, step=0.01, description='Offset')
interval_slider = widgets.FloatSlider(value=0.02, min=0.001, max=0.5, step=0.01, description='Interval')

save_mdc_plot_button = widgets.Button(description="Save Plot as PNG")
save_mdc_plot_button.on_click(lambda _: save_waterfall_plot(waterfall_plot(
    k_x1_slider.value, k_x2_slider.value,
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value,
    offset_slider.value, interval_slider.value,
    k_y_min_slider.value, k_y_max_slider.value
)))

save_mdc_csv_button = widgets.Button(description="Save CSV of Plot")
save_mdc_csv_button.on_click(lambda _: waterfall_plot_save_data(
    k_x1_slider.value, k_x2_slider.value,
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value,
    offset_slider.value, interval_slider.value,
    k_y_min_slider.value, k_y_max_slider.value
))

add_mdc_bounds_button = widgets.Button(description="Add Bounds to CSV")
add_mdc_bounds_button.on_click(lambda _: add_bounds_to_csv_mdc(
    k_x1_slider.value, k_x2_slider.value,
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value, current_date
))

display(widgets.HBox([add_mdc_bounds_button, save_mdc_plot_button, save_mdc_csv_button]))

display(widgets.interact(
    waterfall_plot,
    k_x1=k_x1_slider,
    k_x2=k_x2_slider,
    Ev_Lowerbound=Ev_Lowerbound_slider,
    Ev_Upperbound=Ev_Upperbound_slider,
    offset=offset_slider,
    interval=interval_slider,
    k_y_min=k_y_min_slider,
    k_y_max=k_y_max_slider
))


interactive(children=(FloatSlider(value=-1.0, description='k_x1', max=4.491807287242438, min=-4.50444169591447…

<function __main__.waterfall_plot(k_x1, k_x2, Ev_Lowerbound, Ev_Upperbound, offset, interval, k_y_min, k_y_max)>

CSV data saved successfully to ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_MDC_k_x1_-1.00_k_x2_0.50_E_-1.22_-0.38.csv
